## Import libraries

In [42]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re
import plotly.express as px
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

## Web Scraping Utilities

In [43]:
def init_driver():
    """Initialize headless Chrome driver."""
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.0.0 Safari/537.36")
    return webdriver.Chrome(service=service, options=options)

def split_name_team(player_text):
    """
    Splits a player's name and team from a string like 'James HardenLAC'.
    Returns (name, team).
    """
    suffixes = r"(?:Jr\.|Sr\.|II|III|IV|V)"
    match = re.match(rf"^(.+?)\s*(?:({suffixes}))?\s*([A-Z]{{3}})$", player_text)
    if match:
        player_name = match.group(1).strip()
        suffix = match.group(2) if match.group(2) else ""
        team = match.group(3)
        return f"{player_name} {suffix}".strip(), team
    return player_text, "N/A"

def fetch_nba_stats():
    """
    Uses Selenium to scrape player stats from NBA.com.
    Returns a pandas DataFrame.
    """
    driver = init_driver()
    url = "https://www.nba.com/stats"
    driver.get(url)
    time.sleep(5)

    categories = [
        "Points", "Rebounds", "Assists", "Blocks", "Steals",
        "Turnovers", "Three Pointers Made", "Free Throws Made", "Fantasy Points"
    ]

    data = []
    stat_tables = driver.find_elements(By.CLASS_NAME, "LeaderBoardPlayerCard_lbpcTable__q3iZD")

    for idx, table in enumerate(stat_tables[:len(categories)]):
        rows = table.find_elements(By.CLASS_NAME, "LeaderBoardPlayerCard_lbpcTableRow___Lod5")
        for row in rows:
            columns = row.find_elements(By.TAG_NAME, "td")
            if len(columns) >= 3:
                player_text = columns[1].text
                player_name, team = split_name_team(player_text)
                stat_value = columns[2].text
                data.append([categories[idx], player_name, team, stat_value])

    driver.quit()
    return pd.DataFrame(data, columns=["Category", "Player", "Team", "Stat"])

In [44]:
team_colors = {
    "BOS": "#007A33", "LAL": "#552583", "MIA": "#98002E", "GSW": "#FDB927",
    "CHI": "#CE1141", "NYK": "#006BB6", "PHI": "#006BB6", "BKN": "#000000",
    "MIL": "#00471B", "DAL": "#00538C", "DEN": "#FEC524", "HOU": "#CE1141",
    "OKC": "#007AC1", "SAS": "#C4CED4", "UTA": "#002B5C", "TOR": "#CE1141",
    "NOP": "#0C2340", "IND": "#002D62", "POR": "#E03A3E", "SAC": "#5A2D81",
    "WAS": "#002B5C", "ORL": "#0077C0", "CHA": "#1D1160", "MEM": "#5D76A9",
    "DET": "#C8102E", "MIN": "#78BE20", "ATL": "#E03A3E", "CLE": "#860038",
    "PHX": "#E56020", "LAC": "#C8102E"
}

In [45]:
df = fetch_nba_stats()

In [46]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = "NBA Stats Dashboard"

app.layout = dbc.Container([
    html.H1("NBA Stats Dashboard", className="text-center my-4"),

    dcc.Dropdown(
        id='category-dropdown',
        options=[{'label': cat, 'value': cat} for cat in df['Category'].unique()],
        value=df['Category'].unique()[0],
        clearable=False,
        style={'width': '60%', 'margin': 'auto'}
    ),

    dcc.Graph(id='stats-graph', style={'height': '600px'}),

    dash_table.DataTable(
        id='stats-table',
        columns=[
            {"name": "Player", "id": "Player"},
            {"name": "Team", "id": "Team"},
            {"name": "Stat", "id": "Stat"}
        ],
        style_table={'margin': 'auto', 'width': '80%'},
        style_cell={'textAlign': 'center'},
        page_size=10,
        sort_action="native"
    )
], fluid=True)

In [47]:
@app.callback(
    [Output('stats-graph', 'figure'),
     Output('stats-table', 'data')],
    [Input('category-dropdown', 'value')]
)
def update_dashboard(selected_category):
    """Update bar chart and table based on selected category."""
    filtered_df = df[df['Category'] == selected_category].copy()
    filtered_df["Team"] = filtered_df["Team"].astype(str)
    filtered_df["Stat"] = pd.to_numeric(filtered_df["Stat"], errors="coerce").fillna(0)

    filtered_df['Color'] = filtered_df['Team'].map(lambda team: team_colors.get(team, '#CCCCCC'))

    fig = px.bar(
        filtered_df.sort_values(by="Stat", ascending=True),
        x='Stat', y='Player',
        color='Team',
        text=filtered_df["Stat"].astype(str),
        orientation='h',
        color_discrete_map=team_colors
    )

    fig.update_layout(
        title=f"Top Players in {selected_category}",
        xaxis_title="Stat",
        yaxis_title="Player",
        height=600,
        xaxis=dict(range=[0, filtered_df["Stat"].max() + 5]),
        yaxis={'categoryorder': 'total ascending'}
    )

    return fig, filtered_df.to_dict('records')
    

In [48]:
if __name__ == '__main__':
    app.run(debug=True)